# Leveraging LLMs for Abstract Screening

In [2]:
import json
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, BertModel

from openai import OpenAI
import os
from dotenv import load_dotenv



/workspaces/LLMAbstractScreen/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
Dataset = pd.read_csv('./Dataset/Abstracts_CD008874_20241012_sample.csv')
Examples = pd.read_csv('./Dataset/Abstracts_CD008874_20241012_examples.csv')

In [4]:
def GetInput(row):
    output = '""" ### Title '
    output += row['title']
    output += ' ### Abstract '
    output += row['abstract']
    output += '"""'
    return output

In [5]:
Examples['Input'] = Examples.apply(lambda row: GetInput(row), axis=1)  
Dataset['Input'] = Dataset.apply(lambda row: GetInput(row), axis=1)  

In [6]:
# load BERT tokenizer (uncased)
transformer_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(transformer_name)

# load pretrained BERT model
model = BertModel.from_pretrained(transformer_name)

# assign device (cuda if possible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load to device
model = model.to(device)

In [7]:
def getBertCls(text):
  '''
  this function takes the following input:
  text to be represented by the BERT CLS token
  and gives the following output:
  a numpy array representing the text
  '''
  tok_text = tokenizer(text[:512],
                       return_tensors='pt').to(device)
  mod_output = model(**tok_text,
                     output_hidden_states=True)
  last_hidden_states = mod_output.hidden_states[-1]
  return last_hidden_states[:,0,:].cpu().detach().numpy()[0]

In [8]:
# create column with cls representation of document
Examples['BERT_cls'] = Examples['Input'].apply(getBertCls)
Dataset['BERT_cls'] = Dataset['Input'].apply(getBertCls)


In [9]:
def calCosSim(emb1, emb2):
  '''
  return the cosine similarity of the
  2 input numpy array
  '''
  result = emb1 @ emb2.T
  result /= (np.linalg.norm(emb1)*np.linalg.norm(emb2))
  return result

In [10]:
def GetPosExample(row):
    emb = row['BERT_cls']
    PosExamples = Examples[Examples['Include_cont'] == 1].copy()
    PMID = row['PMID']
    PosExamples = PosExamples[PosExamples['PMID'] != PMID].copy()
    PosExamples['Sim'] = PosExamples['BERT_cls'].apply(lambda x: calCosSim(emb, x))
    sim = PosExamples['Sim']
    
    return PosExamples.iloc[np.argmax(sim)]['Input']    

In [11]:
Examples['PosExample'] = Examples.apply(GetPosExample, axis=1)
Dataset['PosExample'] = Dataset.apply(GetPosExample, axis=1)

In [12]:
# get key
load_dotenv()
client = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY"),
)

In [13]:
prompt = """

### Instruction

You will take the role of a scientific research reviewer to evaluate titles and abstracts for further assessment \
concerning the systematic review study titled \
"Airway physical examination tests for detection of difficult airway management in apparently normal adult patients" \
Please generate a relevance score for the title and abstract in triple quotes based on the following inclusion criteria. \
The relevance score should be between 0 (completely irrelevant) to 100 (perfectly relevant).

### Inclusion Criteria

1. Study Type: Full-text diagnostic test accuracy studies.

2. Index Tests (one or more):
   - Mallampati test
   - Modified Mallampati test
   - Wilson risk score
   - Thyromental distance
   - Sternomental distance
   - Mouth opening test
   - Upper lip bite test

3. Target Condition: Difficult airway, defined by:
   - Difficult face mask ventilation
   - Difficult laryngoscopy
   - Difficult tracheal intubation
   - Failed intubation

4. Participants:
   - Adult patients
   - No obvious airway abnormalities
   - Standard laryngoscope used
   - Standard tracheal tube used


### Output

Expected output format:

{{
    "RelevanceScore": int from 0 to 100,
    "Reason": "Brief explanation of the Relevance Score based on inclusion criteria"
}}

"""


In [14]:
def getRelScore(row):
    txt = row['Input']
    example = row['PosExample']
    prompt_used = prompt + f'\n\n### Example \n\n Below is an example delimited by triple quotes that should definitely be included: \n """{example}"""'
    try:
        completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": prompt_used},
        {"role": "user", "content": txt}
        ]
        )
        return completion.choices[0].message.content
    except:
        return None


In [15]:
Examples['Results'] = Examples.apply(getRelScore, axis=1)


In [16]:

def parseScore(res):
    try:
        js = res[1:-1]
        
        results = json.loads(js)
        
        return results["RelevanceScore"]
    except:
        try:
            js = res[8:-4]
        
            results = json.loads(js)
        
            return results["RelevanceScore"]
        except:
            return "Error"
    



def parseReason(res):
    try:
        js = res[1:-1]
        
        results = json.loads(js)
        
        return results["Reason"]
    except:
        try:
            js = res[8:-4]
        
            results = json.loads(js)
        
            return results["Reason"]
        except:
            return "Error"


In [18]:
Examples['RelevanceScore'] = Examples['Results'].apply(parseScore)    
Examples['Reason'] = Examples['Results'].apply(parseReason)
Examples.to_csv('./Dataset/DynamicOneShot_Training_20241222.csv', index=False)

In [19]:
Dataset['Results'] = Dataset.apply(getRelScore, axis=1)
Dataset['RelevanceScore'] = Dataset['Results'].apply(parseScore)    
Dataset['Reason'] = Dataset['Results'].apply(parseReason)
Dataset.to_csv('./Dataset/DynamicOneShot_dev_20241222.csv', index=False)